In [72]:
from sklearn.datasets import load_breast_cancer

In [73]:
cancer = load_breast_cancer()

In [38]:
from numpy.random import randint
from numpy.random import rand
 
# objective function
def objective(x):
    out = x[0]**2.0 + x[1]**2.0
    return out
 
# decode bitstring to numbers
def decode(bounds, n_bits, bitstring):
    decoded = list()
    largest = 2**n_bits
    for i in range(len(bounds)):
        # extract the substring
        start, end = i * n_bits, (i * n_bits)+n_bits
        substring = bitstring[start:end]
        # convert bitstring to a string of chars
        chars = ''.join([str(s) for s in substring])
        # convert string to integer
        integer = int(chars, 2)
        # scale integer to desired range
        value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
        # store
        decoded.append(value)
    return decoded
 
# tournament selection
def selection(pop, scores, k=3):
    # first random selection
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), k-1):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

# crossover two parents to create two children
def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if rand() < r_cross:
        # select crossover point that is not on the end of the string
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

# mutation operator
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if rand() < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]


In [70]:
# genetic algorithm
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
    # initial population of random bitstring
    pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
    # keep track of best solution
    best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))
    # enumerate generations
                                   
    for gen in range(n_iter):
         # decode population
        decoded = [decode(bounds, n_bits, p) for p in pop]
        # evaluate all candidates in the population
        scores = [objective(d) for d in decoded]
        # check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(">%d, new best f(%s) = %f" % (gen,  decoded[i], scores[i]))
        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        pop = children
    return [best, best_eval]
       

In [71]:
# define range for input
bounds = [[-5.0, 5.0], [-5.0, 5.0]]

# define the total iterations
n_iter = 100

# bits per variable
n_bits = 16

# define the population size
n_pop = 100

# crossover rate
r_cross = 0.9

# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds)) #0.03125
# perform the genetic algorithm search


best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
decoded = decode(bounds, n_bits, best)
print('f(%s) = %f' % (decoded, score))
        

>0, new best f([-0.04852294921875, 0.8917236328125]) = 0.797526
>1, new best f([0.54443359375, 0.413818359375]) = 0.467654
>1, new best f([0.278167724609375, 0.4876708984375]) = 0.315200
>1, new best f([0.46600341796875, -0.230255126953125]) = 0.270177
>2, new best f([0.388336181640625, 0.3314208984375]) = 0.260645
>2, new best f([0.317230224609375, -0.2606201171875]) = 0.168558
>3, new best f([0.374908447265625, 0.021209716796875]) = 0.141006
>4, new best f([0.335845947265625, 0.023345947265625]) = 0.113338
>4, new best f([0.311279296875, 0.101776123046875]) = 0.107253
>4, new best f([0.290679931640625, 0.147247314453125]) = 0.106177
>4, new best f([0.278167724609375, 0.101470947265625]) = 0.087674
>4, new best f([0.13275146484375, -0.2606201171875]) = 0.085546
>5, new best f([0.07568359375, -0.00335693359375]) = 0.005739
>7, new best f([0.0543212890625, 0.006561279296875]) = 0.002994
>10, new best f([0.01708984375, 0.00701904296875]) = 0.000341
>13, new best f([0.0177001953125, 0.004

In [74]:
# testing over breast cancer data
# define range for input
bounds = cancer.data

# define the total iterations
n_iter = 100

# bits per variable
n_bits = 16

# define the population size
n_pop = 100

# crossover rate
r_cross = 0.9

# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds)) #0.00010984182776801405
# perform the genetic algorithm search
best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
decoded = decode(bounds, n_bits, best)
print('f(%s) = %f' % (decoded, score))

>25, new best f([10.437014617919923, 17.9260302734375, 21.213389892578125, 12.94673828125, 18.955843963623046, 13.622630310058593, 19.597444458007814, 16.05841796875, 20.299755859375, 13.567358703613282, 18.5353662109375, 16.688250427246093, 21.206682739257815, 17.897494506835937, 22.287921142578124, 19.3499365234375, 15.840004425048829, 17.558678894042966, 20.341620178222655, 14.2845263671875, 15.157040252685547, 11.113522583007812, 14.950573120117188, 21.430140991210937, 17.52265670776367, 16.76033447265625, 20.859560852050784, 20.225451049804686, 15.310344848632813, 16.40040100097656, 24.231676025390623, 16.454296264648438, 22.494139404296874, 22.0071337890625, 17.524317626953124, 20.16917556762695, 20.399163208007813, 16.615958709716796, 24.04589111328125, 20.033875427246095, 20.885685424804684, 14.37408447265625, 21.460644836425782, 16.72541931152344, 20.630595703125, 18.512597656249998, 11.82857452392578, 15.611498565673829, 13.291262817382814, 15.218903350830079, 19.631279296875

>28, new best f([10.437014617919923, 17.8392138671875, 21.213389892578125, 12.94673828125, 18.955843963623046, 13.622630310058593, 19.597444458007814, 16.05841796875, 20.299755859375, 13.567358703613282, 18.53624755859375, 16.688250427246093, 21.206682739257815, 17.897494506835937, 22.287921142578124, 19.3499365234375, 15.840004425048829, 17.558678894042966, 20.341620178222655, 14.2845263671875, 15.157040252685547, 11.113522583007812, 14.950573120117188, 21.430140991210937, 17.52265670776367, 16.76033447265625, 20.859560852050784, 20.225451049804686, 15.310344848632813, 16.40040100097656, 24.231676025390623, 16.454296264648438, 22.494139404296874, 22.0071337890625, 17.524317626953124, 20.17405990600586, 14.874627685546875, 18.173183135986328, 22.458368225097658, 17.51657440185547, 15.45400909423828, 17.50093994140625, 19.686951904296876, 17.5379345703125, 18.1951904296875, 18.327450561523438, 9.241811096191405, 15.54724639892578, 12.341911315917969, 13.946658477783203, 17.5353845214843

>37, new best f([10.437014617919923, 17.817338867187498, 21.213389892578125, 12.94673828125, 18.955843963623046, 13.622630310058593, 19.597444458007814, 16.05841796875, 20.299755859375, 13.567358703613282, 18.53624755859375, 16.688250427246093, 21.206682739257815, 17.897494506835937, 22.287921142578124, 19.3499365234375, 15.840004425048829, 17.558678894042966, 20.341620178222655, 14.2845263671875, 15.157040252685547, 11.113522583007812, 14.950573120117188, 21.430140991210937, 17.52265670776367, 16.76033447265625, 20.859560852050784, 20.225451049804686, 15.310344848632813, 16.40040100097656, 24.231676025390623, 16.454296264648438, 22.494139404296874, 22.0071337890625, 17.524317626953124, 20.17405990600586, 14.874627685546875, 18.173183135986328, 22.458368225097658, 17.51657440185547, 15.45400909423828, 17.50093994140625, 19.686951904296876, 17.5379345703125, 18.1951904296875, 18.327450561523438, 9.241811096191405, 15.54724639892578, 12.341911315917969, 13.946658477783203, 17.57382202148

>44, new best f([10.437014617919923, 17.79537841796875, 21.213389892578125, 12.94673828125, 18.95602554321289, 13.622630310058593, 19.597444458007814, 16.05841796875, 20.299755859375, 13.657827453613281, 18.53624755859375, 16.688250427246093, 21.206682739257815, 17.897494506835937, 22.287921142578124, 19.3499365234375, 15.840004425048829, 17.558678894042966, 20.341620178222655, 14.2845263671875, 15.157040252685547, 11.113522583007812, 14.950573120117188, 21.430140991210937, 17.52265670776367, 16.76033447265625, 20.859560852050784, 20.225451049804686, 15.310344848632813, 16.40040100097656, 24.231676025390623, 16.454296264648438, 22.494139404296874, 22.0071337890625, 17.524317626953124, 20.17405990600586, 14.874627685546875, 18.173183135986328, 22.458368225097658, 17.51657440185547, 15.45400909423828, 17.50093994140625, 19.686951904296876, 17.5379345703125, 18.1951904296875, 18.327450561523438, 9.241811096191405, 15.54724639892578, 12.341911315917969, 13.946658477783203, 17.5353845214843

>55, new best f([10.437014617919923, 17.795335693359373, 21.213389892578125, 12.94673828125, 18.95602554321289, 13.622630310058593, 19.597444458007814, 16.05841796875, 20.299755859375, 13.657827453613281, 18.53624755859375, 16.688250427246093, 21.206682739257815, 17.897494506835937, 22.287921142578124, 19.3499365234375, 15.840004425048829, 17.558678894042966, 20.341620178222655, 14.2845263671875, 15.157040252685547, 11.113522583007812, 14.950573120117188, 21.430140991210937, 17.52265670776367, 16.76033447265625, 20.859560852050784, 20.225451049804686, 15.310344848632813, 16.40040100097656, 24.231676025390623, 16.454296264648438, 22.494139404296874, 22.0071337890625, 17.524317626953124, 20.17405990600586, 14.874627685546875, 18.173183135986328, 22.458368225097658, 17.51657440185547, 15.45400909423828, 17.50093994140625, 19.688353271484374, 17.5379345703125, 18.1951904296875, 18.327450561523438, 9.241811096191405, 15.54724639892578, 12.341911315917969, 13.946658477783203, 12.615384521484

>59, new best f([10.437014617919923, 17.792601318359374, 21.213389892578125, 12.94673828125, 18.95602554321289, 13.622630310058593, 19.597444458007814, 16.05841796875, 20.299755859375, 13.657827453613281, 18.53624755859375, 16.688250427246093, 21.206682739257815, 17.897494506835937, 22.287921142578124, 19.3499365234375, 15.840004425048829, 17.558678894042966, 20.341620178222655, 14.2845263671875, 15.157040252685547, 11.113522583007812, 14.950573120117188, 21.430140991210937, 17.52265670776367, 16.76033447265625, 20.859560852050784, 20.225451049804686, 15.310344848632813, 16.40040100097656, 24.231676025390623, 16.454296264648438, 22.494139404296874, 22.0071337890625, 17.524317626953124, 20.17405990600586, 14.874627685546875, 18.173183135986328, 22.458368225097658, 17.51657440185547, 15.45400909423828, 17.50093994140625, 19.686951904296876, 17.5379345703125, 18.1951904296875, 18.327450561523438, 9.241811096191405, 15.890371398925781, 12.341911315917969, 13.946658477783203, 17.53538452148

>66, new best f([10.433298797607423, 17.792601318359374, 21.213389892578125, 12.94673828125, 18.95602554321289, 13.622630310058593, 19.597444458007814, 16.05841796875, 20.299755859375, 13.657827453613281, 18.53624755859375, 16.688250427246093, 21.206682739257815, 17.897494506835937, 22.287921142578124, 19.3499365234375, 15.840004425048829, 17.558678894042966, 20.341620178222655, 14.2845263671875, 15.157040252685547, 11.113522583007812, 14.950573120117188, 21.430140991210937, 17.52265670776367, 16.76033447265625, 20.859560852050784, 20.225451049804686, 15.310344848632813, 16.40040100097656, 24.231676025390623, 16.454296264648438, 22.494139404296874, 22.0071337890625, 17.524317626953124, 20.17405990600586, 14.874627685546875, 18.173183135986328, 22.458368225097658, 17.51657440185547, 15.45400909423828, 17.50093994140625, 19.686951904296876, 17.5379345703125, 18.1951904296875, 18.327450561523438, 9.241811096191405, 15.890371398925781, 12.341911315917969, 13.946658477783203, 17.53538452148

>72, new best f([10.425867156982424, 17.792601318359374, 21.213389892578125, 12.94673828125, 18.95602554321289, 13.622630310058593, 19.597444458007814, 16.05841796875, 20.299755859375, 13.657827453613281, 18.53624755859375, 16.688250427246093, 21.206682739257815, 17.897494506835937, 22.287921142578124, 19.3499365234375, 15.840004425048829, 17.558678894042966, 20.341620178222655, 14.2845263671875, 15.157040252685547, 11.113522583007812, 14.950573120117188, 21.430140991210937, 17.52265670776367, 16.76033447265625, 20.859560852050784, 20.225451049804686, 15.310344848632813, 16.40040100097656, 24.231676025390623, 16.454296264648438, 22.494139404296874, 22.0071337890625, 17.524317626953124, 20.17405990600586, 14.874627685546875, 18.173183135986328, 22.458368225097658, 17.51657440185547, 15.45400909423828, 17.50093994140625, 19.686951904296876, 17.5379345703125, 18.1951904296875, 18.327450561523438, 9.241811096191405, 15.890371398925781, 12.341911315917969, 13.946658477783203, 17.53538452148

Done!
f([10.425867156982424, 17.792601318359374, 21.213389892578125, 12.94673828125, 18.95602554321289, 13.622630310058593, 19.597444458007814, 16.05841796875, 20.299755859375, 13.657827453613281, 18.53624755859375, 16.688250427246093, 21.206682739257815, 17.897494506835937, 22.287921142578124, 19.3499365234375, 15.840004425048829, 17.558678894042966, 20.341620178222655, 14.2845263671875, 15.157040252685547, 11.113522583007812, 14.950573120117188, 21.430140991210937, 17.52265670776367, 16.76033447265625, 20.859560852050784, 20.225451049804686, 15.310344848632813, 16.40040100097656, 24.231676025390623, 16.454296264648438, 22.494139404296874, 22.0071337890625, 17.524317626953124, 20.17405990600586, 14.874627685546875, 18.173183135986328, 22.458368225097658, 17.51657440185547, 15.45400909423828, 17.50093994140625, 19.686951904296876, 17.5379345703125, 18.1951904296875, 18.327450561523438, 9.241811096191405, 15.890371398925781, 12.341911315917969, 13.946658477783203, 17.535384521484374, 14

In [81]:
print(decoded[568])

16.43318298339844


In [83]:
print(best)

[1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 

In [84]:
print(r_mut)

0.00010984182776801405
